In [9]:
import json
import pandas as pd
import re
import random
from ast import literal_eval as make_tuple
from scipy import sparse
import numpy as np
from pymongo import MongoClient
from nltk.corpus import stopwords
from sklearn import svm
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn_deltatfidf import DeltaTfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn import svm
from sklearn.metrics import accuracy_score
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import precision_score
from sklearn.metrics import log_loss
from sklearn.feature_extraction.text import TfidfVectorizer
import xgboost as xgb
import sys
sys.path.append('../machine_learning')
import yelp_ml as yml
reload(yml)
from gensim import corpora, models, similarities, matutils
import tqdm

In [10]:
#Import scrapped reviews
dc_reviews = json.load(open("../Yelp_web_scrapper/dc_reviews.json"))
newyork_reviews = json.load(open("../Yelp_web_scrapper/newyork_reviews.json"))

In [11]:
#Import word dictionaries
lh_neg = open('../input/negative-words.txt', 'r').read()
lh_neg = lh_neg.split('\n')
lh_pos = open('../input/positive-words.txt', 'r').read()
lh_pos = lh_pos.split('\n')
users = json.load(open("cleaned_large_user_dictionary.json"))
word_list = list(set(lh_pos + lh_neg))

In [12]:
large_user_number = 76

In [ ]:
#####Pull restaurant data for a given user
ip = '54.175.170.119'
conn = MongoClient(ip, 27017)
conn.database_names()
db = conn.get_database('cleaned_data')
reviews = db.get_collection('restaurant_reviews')

for j in tqdm.tqdm(range(70, 100)):
    user_df = yml.make_user_df(users[users.keys()[j]])
    business_ids = list(set(user_df['biz_id']))
    restreview = {}

    for i in range(0, len(business_ids)):
        rlist = []
        for obj in reviews.find({'business_id':business_ids[i]}):
            rlist.append(obj)
        restreview[business_ids[i]] = rlist
    restaurant_df = yml.make_biz_df(users.keys()[j], restreview)
    
    #Create a training and test sample from the user reviewed restaurants
    split_samp = .25
    len_random = int(len(business_ids) * split_samp)
    test_set = random.sample(business_ids, len_random)
    training_set = [x for x in business_ids if x not in test_set]
    sub_train_reviews, train_labels, train_reviews, train_ratings = [], [], [], []

    #Create a list of training reviews and training ratings
    for rest_id in training_set:
        train_reviews.append((user_df[user_df['biz_id'] == rest_id]['review_text'].iloc[0],
                                 user_df[user_df['biz_id'] == rest_id]['rating'].iloc[0]))

    #Create an even sample s.t. len(positive_reviews) = len(negative_reviews)
    sample_size = min(len([x[1] for x in train_reviews if x[1] < 4]),
                          len([x[1] for x in train_reviews if x[1] >= 4]))
    
    bad_reviews = [x for x in train_reviews if x[1] < 4]
    good_reviews = [x for x in train_reviews if x[1] >= 4]

    for L in range(0, int(float(sample_size)/float(2))):
        sub_train_reviews.append(bad_reviews[L][0])
        sub_train_reviews.append(good_reviews[L][0])
        train_labels.append(bad_reviews[L][1])
        train_labels.append(good_reviews[L][1])
        
    #Make the train labels binary
    train_labels = [1 if x >=4 else 0 for x in train_labels]
    
    #Fit LSI model and return number of LSI topics
    lsi, topics, dictionary = yml.fit_lsi(sub_train_reviews)

    #Make a FeatureUnion object with the desired features then fit to train reviews
    test_results = {}
    feature_selection = {"sent_tf":(True, True, False), 
                         "sent": (True,False,False),
                         "tf_lda": (False,True,True), 
                         "all": (True, True, True)}

    for feature in feature_selection.keys():
        #Make a FeatureUnion object with the desired features then fit to train reviews
        comb_features = yml.make_featureunion(sent_percent=feature_selection[feature][0], 
                                              tf = feature_selection[feature][1], 
                                              lda = feature_selection[feature][2])
        
        delta_vect = None
        comb_features.fit(sub_train_reviews)
        train_features = comb_features.transform(sub_train_reviews)
        train_lsi = yml.get_lsi_features(sub_train_reviews, lsi, topics, dictionary)
        train_features = sparse.hstack((train_features, train_lsi))
        train_features = train_features.todense()

        #Fit LSI model and return number of LSI topics
        lsi, topics, dictionary = yml.fit_lsi(sub_train_reviews)
        
        #fit each model in turn 
        model_runs = {"svm": (True, False, False),
                      "rf": (False, True, False), 
                      "naive_bayes": (False, False, True)}

        for model_run in model_runs.keys():
            clf = yml.fit_model(train_features, train_labels, svm_clf = model_runs[model_run][0], 
                            RandomForest = model_runs[model_run][1], 
                                nb = model_runs[model_run][2])
            threshold = 0.7
            error = yml.test_user_set(test_set, clf, restaurant_df, user_df, comb_features, 
                                      threshold, lsi, topics, dictionary, delta_vect)
            test_results[(feature, model_run)] = (yml.get_log_loss(error), 
                                            yml.get_accuracy_score(error), 
                                            yml.get_precision_score(error))
    
    string_keys_dict = {}
    for key in test_results.keys():
        string_keys_dict[str(key)] = test_results[key]
            
with open('test_results.json', 'wb') as fp:
    json.dump(string_keys_dict, fp)

  0%|          | 0/30 [00:00<?, ?it/s]

In [14]:
test_results = string_keys_dict.copy()

In [15]:
test_results

{"('all', 'naive_bayes')": (-0.35531558821035314, 0.49295774647887325, 0.5),
 "('all', 'rf')": (-0.8133847697787171, 0.49295774647887325, 0.5),
 "('all', 'svm')": (-0.8399640423809434,
  0.36619718309859156,
  0.6046511627906976),
 "('sent', 'naive_bayes')": (-0.9427207768388881,
  0.5492957746478874,
  0.46296296296296297),
 "('sent', 'rf')": (-0.03605482107092158, 0.49295774647887325, 0.0),
 "('sent', 'svm')": (-0.27756518605734753, 0.49295774647887325, 0.5),
 "('sent_tf', 'naive_bayes')": (-0.448129757136043, 0.49295774647887325, 0.5),
 "('sent_tf', 'rf')": (-0.16551179418463383, 0.49295774647887325, 0.0),
 "('sent_tf', 'svm')": (-0.44496246323770344, 0.49295774647887325, 0.5),
 "('tf_lda', 'naive_bayes')": (-0.44581605225654675, 0.49295774647887325, 0.5),
 "('tf_lda', 'rf')": (-0.4093586504561011, 0.49295774647887325, 0.5),
 "('tf_lda', 'svm')": (-0.6585684158330528,
  0.5492957746478874,
  0.4166666666666667)}

In [16]:
#########################
#Make a Recommendation
#########################
top_results = []
#Get feature and model combination that yields the highest precision
for key in test_results.keys():
    feat_model = make_tuple(key)
    if not top_results:
        top_results = [(feat_model,test_results[key][2])]
    else:
        if test_results[key][2] > top_results[0][1]:
            top_results.pop()
            top_results = [(feat_model, test_results[key][2])]
feat_result = top_results[0][0][0]
model_result = top_results[0][0][1]

for j in tqdm.tqdm(range(large_user_number, large_user_number+1)):
    user_df = yml.make_user_df(users[users.keys()[j]])
    business_ids = list(set(user_df['biz_id']))

    #Create a list of training reviews and training ratings
    for rest_id in business_ids:
        train_reviews.append((user_df[user_df['biz_id'] == rest_id]['review_text'].iloc[0],
                                 user_df[user_df['biz_id'] == rest_id]['rating'].iloc[0]))

    #Create an even sample s.t. len(positive_reviews) = len(negative_reviews)
    sample_size = min(len([x[1] for x in train_reviews if x[1] < 4]),
                          len([x[1] for x in train_reviews if x[1] >= 4]))
    
    bad_reviews = [x for x in train_reviews if x[1] < 4]
    good_reviews = [x for x in train_reviews if x[1] >= 4]
    
    train_labels = []
    sub_train_reviews = []
    for L in range(0, int(float(sample_size)/float(2))):
        sub_train_reviews.append(bad_reviews[L][0])
        sub_train_reviews.append(good_reviews[L][0])
        train_labels.append(bad_reviews[L][1])
        train_labels.append(good_reviews[L][1])
        
    #Make the train labels binary
    train_labels = [1 if x >=4 else 0 for x in train_labels]
    
    #Fit LSI model and return number of LSI topics
    lsi, topics, dictionary = yml.fit_lsi(sub_train_reviews)

    #Make a FeatureUnion object with the desired features then fit to train reviews
    feature_selection = {"sent_tf":(True, True, False), 
                         "sent": (True,False,False),
                         "tf_lda": (False,True,True), 
                         "all": (True, True, True)}
    top_feature = feature_selection['all']
    
    comb_features = yml.make_featureunion(sent_percent=top_feature[0], 
                                          tf = top_feature[1], 
                                          lda = top_feature[2])
        
    comb_features.fit(sub_train_reviews)
    train_features = comb_features.transform(sub_train_reviews)
    train_lsi = yml.get_lsi_features(sub_train_reviews, lsi, topics, dictionary)
    train_features = sparse.hstack((train_features, train_lsi))
    train_features = train_features.todense()

    #Fit LSI model and return number of LSI topics
    lsi, topics, dictionary = yml.fit_lsi(sub_train_reviews)
        
    #Get the top performing model and fit using that model
    model_runs = {"svm": (True, False, False),
                  "rf": (False, True, False), 
                  "naive_bayes": (False, False, True)}
    top_model = model_runs['svm']
    clf = yml.fit_model(train_features, train_labels, svm_clf = top_model[0], 
                RandomForest = top_model[1], 
                    nb = top_model[2])

    threshold = 0.7
    user_results = yml.make_rec(dc_reviews, clf, threshold, comb_features, 
                                lsi, topics, dictionary)

  0%|          | 0/1 [00:00<?, ?it/s]/Users/robertsonwang/anaconda2/lib/python2.7/site-packages/sklearn/decomposition/online_lda.py:508: DeprecationWarning: The default value for 'learning_method' will be changed from 'online' to 'batch' in the release 0.20. This warning was introduced in 0.18.
  DeprecationWarning)
/Users/robertsonwang/anaconda2/lib/python2.7/site-packages/sklearn/feature_extraction/text.py:764: UnicodeWarning: Unicode equal comparison failed to convert both arguments to Unicode - interpreting them as being unequal
  feature_idx = vocabulary[feature]
100%|██████████| 1/1 [00:39<00:00, 39.66s/it]


In [17]:
################################################################
#Collect the results into a list of tuples, then select the top
#5 most confident good recs and top 5 most confident bad recs
################################################################

tuple_results = []
for result in user_results:
    tuple_results.append((result[1], result[2], result[3]))
    
#Sort the list of tuples by predicition confidence
tuple_results = sorted(tuple_results, key=lambda tup: tup[1])
top_5 = tuple_results[-5:]
bottom_5 = tuple_results[0:5]

In [18]:
#Show the top 5 restaurants
top_5

[(u'/biz/potters-house-washington', 0.90000000000000002, 1),
 (u'/biz/rice-bar-washington-4', 0.90000000000000002, 1),
 (u'/biz/that-cheesecake-truck-washington-2', 0.90000000000000002, 1),
 (u'/biz/little-sesame-washington', 0.90000000000000002, 1),
 (u'/biz/ching-ching-cha-washington', 0.94999999999999996, 1)]

In [19]:
#Show the bottom 5 restaurants
bottom_5

[(u'/biz/lukes-lobster-georgetown-washington', 0.050000000000000003, 0),
 (u'/biz/good-stuff-eatery-washington-2', 0.050000000000000003, 0),
 (u'/biz/bold-bite-washington', 0.050000000000000003, 0),
 (u'/biz/macintyres-washington', 0.14999999999999999, 0),
 (u'/biz/takoda-restaurant-and-beer-garden-washington-2',
  0.14999999999999999,
  0)]

In [20]:
#user_df contains all of the user's reviews, with one column of restaurants,
#one column of the user's ratings, and one column with the user's reviews
user_df.head()

,biz_id,rating,review_text
0,24f5ie972m_bYpirVW-dZA,4,As far as international pizza chains goes Roun...
1,1lRytuj8IdLMHDFl2LetxA,1,DO THIS FIRSTSign up and get the Club Palms ca...
2,G-O_a7168Id7E0wIgHKtVw,3,I had my doubts about this new Thai restaurant...
3,FVospBWddyS7IpbEuFSqgQ,2,Maybe my expectations were higher than normal ...
4,4I4OoW0tbaoG4I4D2spF3w,4,Del Taco insert greater than sign Taco Bell ...
